# Python для анализа данных

## Библиотеки для работы с данными в табличном формате: pandas. SQL для Python. Работа с Clickhouse. 

Автор: *Ян Пиле, НИУ ВШЭ*

Мы с вами уже немного поработали с запросами данных из интернета, причем как непосредственно с сайтов, так и через некоторые API. Давайте теперь попробуем поработать с SQL прямо из Python.

### Порешаем задачи

In [1]:
import json # Чтобы разбирать поля
import requests # чтобы отправлять запрос к базе
import pandas as pd # чтобы в табличном виде хранить результаты запроса

Нужно написать функцию, которая будет отправлять текст SQL-запроса, в нашем случае - запроса к ClickHouse, на сервер, а по выполнении запроса забирать его результаты в каком-то виде.

In [2]:
# имена явки пароли. если хотите, чтобы считалось с вашего логина, вставьте сюда свои логин и пароль
USER = ''
PASS = ''
HOST = 'http://hse.beslan.pro:8080/'

def get_clickhouse_data(query,
                        host=HOST, 
                        USER = USER, 
                        PASS = PASS, 
                        connection_timeout = 1500, 
                        dictify=True, 
                        **kwargs):
    NUMBER_OF_TRIES = 5  # Количество попыток запуска
    DELAY = 10           #время ожидания между запусками
    import time
    params = kwargs      #если вдруг нам нужно в функцию положить какие-то параметры
    if dictify:
        query += "\n FORMAT JSONEachRow"   # dictify = True отдает каждую строку в виде JSON'a

    for i in range(NUMBER_OF_TRIES):

        headers = {'Accept-Encoding': 'gzip'}

        r = requests.post(host, 
                          params = params, 
                          auth=(USER, PASS), 
                          timeout = connection_timeout, 
                          data=query
                          )              # отправили запрос на сервер

        if r.status_code == 200 and not dictify:    
            return r.iter_lines()         # генератор :)
        elif r.status_code == 200 and dictify:
            return (json.loads(x) for x in r.iter_lines()) # генератор :)
        
        else:
            print('ATTENTION: try #%d failed' % i)
            if i != (NUMBER_OF_TRIES - 1):
                print(r.text)
                time.sleep(DELAY * (i + 1))
            else:
                raise(ValueError, r.text)

Функция ниже преобразует полученные нами данные из генератора в pd.Dataframe

In [10]:
query = """
select  *
from default.events
limit 10
"""

In [11]:
d = get_clickhouse_data(query, dictify=True)

In [12]:
next(d)

{'AppPlatform': 'android',
 'events': 8,
 'EventDate': '2019-09-29',
 'DeviceID': '7429291373250434008'}

In [3]:
def get_data(query):
    return pd.DataFrame(list(get_clickhouse_data(query, dictify=True)))

In [22]:
get_data(query)

,AppPlatform,events,EventDate,DeviceID
0,android,8,2019-09-29,7429291373250434008
1,android,175,2019-09-15,7429291824672902510
2,android,0,2019-09-17,7429291824672902510
3,android,0,2019-09-26,7429291824672902510
4,android,4,2019-04-29,7429292273953361459
5,android,38,2019-08-20,7429293114537639018
6,android,38,2019-05-21,7429298825563999474
7,android,4,2019-05-26,7429298825563999474
8,android,100,2019-08-07,7429300397574411770
9,android,26,2019-01-31,7429301272237917347


Предлагаю немного разобраться в структуре нашей базы. Давайте достанем по 5-10 строк каждой из таблиц и посмотри, что же в них лежит. В events, например, уложены AppPlatform - Платформа (операционная система мобильного устройства), events - количество событий, произошедших в эту дату (будем, например, считать, что события это клики и каждый из них платный), EventDate - Дата события, DeviceID - идентификатор устройства.

In [9]:
# впуливаем сюда запрос
query = """
select  *
from default.events
limit 10
"""

In [29]:
f = get_data(query)

In [30]:
f

,AppPlatform,events,EventDate,DeviceID
0,android,8,2019-09-29,7429291373250434008
1,android,175,2019-09-15,7429291824672902510
2,android,0,2019-09-17,7429291824672902510
3,android,0,2019-09-26,7429291824672902510
4,android,4,2019-04-29,7429292273953361459
5,android,38,2019-08-20,7429293114537639018
6,android,38,2019-05-21,7429298825563999474
7,android,4,2019-05-26,7429298825563999474
8,android,100,2019-08-07,7429300397574411770
9,android,26,2019-01-31,7429301272237917347


Только что мы научились превращать результат нашего SQL-запроса в PANDAS-dataframe.

В devices, например, уложены UserID - идентификатор пользователя, DeviceID - идентификатор устройства.

In [10]:
query = """
SELECT *
from devices
limit 10
"""
get_data(query)

,DeviceID,UserID
0,290132773793,9407952136059258036
1,619578718457,14561372283986042425
2,9011245721293,284698082657182871
3,11452758091056,17260156507956968854
4,12397387173463,4870696193129182034
5,12599755717733,6758018588637972435
6,13210761921520,3955806493898881684
7,15216677602256,15186450329800833643
8,16775400460803,5591788874343608413
9,21701379700142,14176532582507529800


Проверим, однозначное ли это соответствие (может ли у человека быть два устройства и могут ли с одного устройства сидеть два человека)

In [12]:
query = """
SELECT UserID, uniqExact(DeviceID) as cnt
from devices
group by UserID
having cnt>1
"""
get_data(query)

""


In [13]:
query = """
SELECT DeviceID, uniqExact(UserID) as cnt
from devices
group by DeviceID
having cnt>1
"""
get_data(query)

""


Видим, что оба запроса возвращают пустой результат. Это означает, что соответствие между UserID и DeviceID взаимно-однозначное. Это позволит нам избежать многих проблем впоследствии. 

В checks хранится стоимость всех покупок одного UserID за день, BuyDate - дата покупки, Rub - стоимость покупки

In [19]:
query = """
SELECT *
from checks
limit 10
"""
get_data(query)

,Rub,BuyDate,UserID
0,3,2019-10-04,18446583642950580515
1,4,2019-10-04,18446535622689003675
2,0,2019-10-04,18446130411954852964
3,3,2019-10-04,18446003252714243011
4,8,2019-10-04,18445948434655311802
5,17,2019-10-04,18445927732647659917
6,2,2019-10-04,18445761122620052505
7,15,2019-10-04,18445655133428855896
8,1,2019-10-04,18445587876544434519
9,1,2019-10-04,18445586451093345117


Проверим, есть ли записи, у которых набору UserID-BuyDate соответствует несколько записей

In [20]:
query = """
SELECT BuyDate, UserID, count(*) as cnt
from checks
group by BuyDate, UserID
having cnt>1
"""
get_data(query)

""


Нам снова повезло! На каждого человека и каждый день в таблицу пишется суммарная стоимость его покупок. Теперь посмотрим на таблицу installs. 

В ней InstallationDate - дата установки, InstallCost - стоимость установки, Platform - Платформа (операционная система мобильного устройства), DeviceID - идентификатор устройства и Source - источник трафика (откуда человек пришел устанавливать приложение: сам нашел в поисковике, из рекламы, перешел по реферальной ссылке и т.д.)

In [22]:
query = """
SELECT *
from installs
limit 10
"""
get_data(query)

,InstallationDate,InstallCost,Platform,DeviceID,Source
0,2019-03-02,0,android,7950068545577019282,Source_27
1,2019-03-17,49,android,17173992779193729517,Source_14
2,2019-04-07,56,android,9528182466778893591,Source_14
3,2019-06-25,39,android,2212531864415574595,Source_9
4,2019-04-13,0,android,6959033924999748551,Source_27
5,2019-01-27,0,android,262831700047781552,Source_27
6,2019-07-16,0,android,8026675461764302030,Source_27
7,2019-09-19,0,android,14556943550242489445,Source_27
8,2019-03-31,0,iOS,2367522355246575108,Source_27
9,2019-02-14,0,iOS,7134503170474894060,Source_27


Давайте сформулируем несколько задач, которые мы на этих данных хотим решить.

### В течение какого срока установка, в среднем, окупается, в зависимости от:
* платформы 
* источника трафика 

Воспользуйтесь тем периодом данных, который сочтете обоснованным для формулировки вывода

Давайте посмотрим среднюю стоимость установки в зависимости от источника трафика. Будем считать, что GMV мы считаем в валюте Rub\*5, а стоимость одного события равна 0.5 у.е.

Для начала достанем информацию, которая касается установок приложения и приклеим к ней информацию о том, какой UserID какие установки совершил. Для этого надо сделать join таблиц installs и devices. Я предлагаю считать данные за 1 квартал 2019 (почему бы и нет). В отображении остаывим 10 записей, чтобы экран не заполнять лишним. Join сделаем inner, предполагая, что нет таких DeviceID, которые никому не принадлежат (хотя вообще говоря, это стоит проверить)

In [4]:
query = """
select a.Source as Source, 
    a.InstallationDate as InstallationDate, 
    a.InstallCost as InstallCost, 
    a.DeviceID as DeviceID,
    b.UserID as UserID
from installs as a
inner join devices as b
on a.DeviceID = b.DeviceID
where InstallationDate between '2019-01-01' and '2019-03-31'
limit 10
"""

res = get_data(query)
res

,Source,InstallationDate,InstallCost,DeviceID,UserID
0,Source_14,2019-03-17,49,17173992779193729517,11414413802604236781
1,Source_27,2019-03-31,0,2367522355246575108,14078818521557937906
2,Source_27,2019-02-14,0,7134503170474894060,16277346519442177193
3,Source_27,2019-02-09,0,10818712337819873473,17267778333827433349
4,Source_14,2019-03-05,192,16757737255390617386,16608727387252734769
5,Source_27,2019-02-17,0,7221474580778093168,12085018548422553596
6,Source_27,2019-01-13,0,1162593927025570626,15858290979918448277
7,Source_27,2019-02-13,0,16729626202875967956,635431576042657930
8,Source_9,2019-01-05,191,80176470024453353,9618826834457433137
9,Source_27,2019-01-22,0,6960243446572445433,3108403965972211033


Теперь нам нужно посчитать суммарную стоимость всех заказов, которые указанный UserID сделал за этот квартал (это как раз один из двух источников нашей доходной части), а расходную часть - InstallCost- мы уже достали. Здесь необходимо делать left join, потому что могут быть люди, которые ничего не купили за этот период, хоть и установили приложение. Значит наше условие ограничения на left join должно брать только те покупки людей, которые произошли от даты установки до конца квартала, а также оставлять записи, в которых не было ни одной покупки, это можно обеспечить условием BuyDate is null (в правой таблице не нашлось ни одной покупки). После того, как мы эту информацию приджойнили, посчитаем на каждый факт установки суммарную стоимость всех покупок с помощью функции sum(). Мы также хотим, чтоб при суммировании у тех, кто не купил ничего в поле GMV - Gross Merchandise Value (суммарный оборот заказов)- стоял ноль. Для этого мы сначала переведем содержимое поля Rub в интересующую нас валюту (мы договорились умножать его на 5), а потом суммировать не само получившееся значение, а coalesce(Rub\*5,0) эта функция возвращает первое непустое значение из списка своих аргументов. Получается, что если поле Rub заполнено, она вернет Rub\*5, а если человек ничего не купил, то она вернет 0, как раз этого мы и добивались. Стоит заметить, что в качестве левой таблицы для join'а мы вставили наш предыдущий запрос.

In [6]:
query = """
select a.Source as Source, 
        a.InstallationDate as InstallationDate, 
        a.InstallCost as InstallCost, 
        a.DeviceID as DeviceID,
        b.UserID as UserID,
        sum(coalesce(b.Rub*5, 0)) as GMV
from  (select a.Source as Source, 
        a.InstallationDate as InstallationDate, 
        a.InstallCost as InstallCost, 
        a.DeviceID as DeviceID,
        b.UserID as UserID
    from installs as a
    inner join devices as b
    on a.DeviceID = b.DeviceID
    where InstallationDate between '2019-01-01' and '2019-03-31') as a
left join checks as b
    on a.UserID = b.UserID
where (b.BuyDate >= a.InstallationDate
    and b.BuyDate<='2019-03-31')
    or b.BuyDate is null
group by a.Source , 
        a.InstallationDate, 
        a.InstallCost, 
        a.DeviceID,
        b.UserID
limit 10
"""

res = get_data(query)
res

,Source,InstallationDate,InstallCost,DeviceID,UserID,GMV
0,Source_27,2019-02-16,0,15192206093305513656,8920789046885116335,0
1,Source_14,2019-02-27,188,12689407752654771346,524826333953903216,20
2,Source_27,2019-02-05,0,1107212526051331520,18128912033971760883,110
3,Source_9,2019-01-04,290,11393135978085233130,13353316700753764690,130
4,Source_27,2019-02-15,0,17402265388963424266,16665079535259736297,40
5,Source_9,2019-02-17,295,11192981545437806453,13022903700187538270,50
6,Source_27,2019-02-17,0,6679132803466357599,17224259138483230813,195
7,Source_14,2019-03-10,217,10772887830696022577,18038729512940335358,525
8,Source_27,2019-01-03,0,11450573820091658738,14883591389812706776,3015
9,Source_14,2019-03-22,335,12081178027465767903,9110835495485449673,0


Остается предпоследний шаг: таким же образом собрать информацию о произошедших событиях (они лежат в поле events таблицы events и мы договорились, что стоимость одного события - 0.5 у.е.). Полностью повторим логику, которая у нас была до этого. Только в этот раз попробуем в функцию sum() не подставлять coalesce. Если мы уверены, что в каждом Source произошло хотя бы одно событие, то в итоговом результате наша сумма будет точно ненулевой. 

In [7]:
query = """
select a.Source as Source, 
            a.InstallationDate as InstallationDate, 
            a.InstallCost as InstallCost, 
            a.DeviceID as DeviceID,
            a.UserID as UserID,
            a.GMV as GMV,
            sum(events*0.5) as events_revenue
from   (select a.Source as Source, 
            a.InstallationDate as InstallationDate, 
            a.InstallCost as InstallCost, 
            a.DeviceID as DeviceID,
            b.UserID as UserID,
            sum(coalesce(b.Rub*5, 0)) as GMV
    from  (select a.Source as Source, 
            a.InstallationDate as InstallationDate, 
            a.InstallCost as InstallCost, 
            a.DeviceID as DeviceID,
            b.UserID as UserID
        from installs as a
        inner join devices as b
        on a.DeviceID = b.DeviceID
        where InstallationDate between '2019-01-01' and '2019-03-31') as a
    left join checks as b
        on a.UserID = b.UserID
    where (b.BuyDate >= a.InstallationDate
        and b.BuyDate<='2019-03-31')
        or b.BuyDate is null
    group by a.Source , 
            a.InstallationDate, 
            a.InstallCost, 
            a.DeviceID,
            b.UserID) as a
left join events as b
on a.DeviceID = b.DeviceID
where (b.EventDate >= a.InstallationDate
        and b.EventDate<='2019-03-31')
        or b.EventDate is null
group by a.Source as Source, 
            a.InstallationDate as InstallationDate, 
            a.InstallCost as InstallCost, 
            a.DeviceID as DeviceID,
            a.UserID as UserID,
            a.GMV as GMV
limit 10
"""

res = get_data(query)
res

,Source,InstallationDate,InstallCost,DeviceID,UserID,GMV,events_revenue
0,Source_14,2019-02-26,249,444140439363382884,11003954550335868251,15,76.5
1,Source_27,2019-03-03,0,4354043079275935996,1606494396501356622,205,89.5
2,Source_9,2019-02-24,154,15977104374208615893,6096895903643675828,290,29.5
3,Source_9,2019-02-15,112,3053694705143773856,14891844260302256718,30,20.5
4,Source_9,2019-03-13,93,5304606943218537309,14507677921561174828,80,13.0
5,Source_9,2019-01-08,119,7737784539216938344,4764661857079284967,370,250.0
6,Source_9,2019-03-28,183,7716318740094014842,11878190050455408874,15,7.0
7,Source_27,2019-02-24,0,8295368433497218728,5091211158312248090,135,114.5
8,Source_9,2019-03-14,97,2168443150386895127,471574737188762008,30,14.5
9,Source_9,2019-02-25,9,7101761698018637152,5490547953140710855,0,66.0


Ну и теперь произведем финальный шаг: суммируем все по источникам трафика и сразу посчитаем ROI - суммарный доход/суммарные затраты

In [8]:
query = """

select Source, uniqExact(UserID) as users,
    SUM(InstallCost) AS InstallCost,
    sum(GMV) as GMV,
    SUM(events_revenue) AS events_revenue
from  (select a.Source as Source, 
                a.InstallationDate as InstallationDate, 
                a.InstallCost as InstallCost, 
                a.DeviceID as DeviceID,
                a.UserID as UserID,
                a.GMV as GMV,
                sum(events*0.5) as events_revenue
    from   (select a.Source as Source, 
                a.InstallationDate as InstallationDate, 
                a.InstallCost as InstallCost, 
                a.DeviceID as DeviceID,
                b.UserID as UserID,
                sum(coalesce(b.Rub*5, 0)) as GMV
        from  (select a.Source as Source, 
                a.InstallationDate as InstallationDate, 
                a.InstallCost as InstallCost, 
                a.DeviceID as DeviceID,
                b.UserID as UserID
            from installs as a
            inner join devices as b
            on a.DeviceID = b.DeviceID
            where InstallationDate between '2019-01-01' and '2019-03-31') as a
        left join checks as b
            on a.UserID = b.UserID
        where (b.BuyDate >= a.InstallationDate
            and b.BuyDate<='2019-03-31')
            or b.BuyDate is null
        group by a.Source , 
                a.InstallationDate, 
                a.InstallCost, 
                a.DeviceID,
                b.UserID) as a
    left join events as b
    on a.DeviceID = b.DeviceID
    where (b.EventDate >= a.InstallationDate
            and b.EventDate<='2019-03-31')
            or b.EventDate is null
    group by a.Source as Source, 
                a.InstallationDate as InstallationDate, 
                a.InstallCost as InstallCost, 
                a.DeviceID as DeviceID,
                a.UserID as UserID,
                a.GMV as GMV
    )
group by Source
"""
res = get_data(query)
res

,Source,users,InstallCost,GMV,events_revenue
0,Source_7,5152,0,1279815,813468.0
1,Source_6,4,391,165,36.5
2,Source_9,360783,63012041,53098015,26022904.0
3,Source_5,10581,1848358,2263355,1348541.5
4,Source_1,4,749,245,583.0
5,Source_14,150296,26245642,31349710,13385514.5
6,Source_25,633,0,157655,105314.5
7,Source_12,1,231,5,5.0
8,Source_15,26990,4701355,5218430,2047272.5
9,Source_24,2,0,935,331.5


С помощью pandas приведем поля к нужному нам формату (По умолчанию Clickhouse выплевывает результаты в строковом формате)

In [9]:
res = res.astype({'users':int, 'InstallCost':float, 'GMV':float, 'events_revenue':float})

Также посчитаем доходную часть

In [17]:
res['Profit'] = res['GMV'] + res['events_revenue']

И, наконец, посчитаем ROI

In [21]:
res['ROI'] = res['Profit']/res['InstallCost']
res

,Source,users,InstallCost,GMV,events_revenue,Profit,ROI
0,Source_7,5152,0.0,1279815.0,813468.0,2093283.0,inf
1,Source_6,4,391.0,165.0,36.5,201.5,0.515345
2,Source_9,360783,63012041.0,53098015.0,26022904.0,79120919.0,1.255648
3,Source_5,10581,1848358.0,2263355.0,1348541.5,3611896.5,1.954111
4,Source_1,4,749.0,245.0,583.0,828.0,1.105474
5,Source_14,150296,26245642.0,31349710.0,13385514.5,44735224.5,1.704482
6,Source_25,633,0.0,157655.0,105314.5,262969.5,inf
7,Source_12,1,231.0,5.0,5.0,10.0,0.043290
8,Source_15,26990,4701355.0,5218430.0,2047272.5,7265702.5,1.545449
9,Source_24,2,0.0,935.0,331.5,1266.5,inf


Что мы видим на примере данных:
    
    1. Бесплатные каналы привлечения приносят большую часть наш доходов
    2. Среди платных каналов не окупилось всего два, да и то крайне малые.
    3. Крупные платные каналы Source9 и Source14 имеют сильно отличающийся ROI - можно поразбираться, почему так
    
Аналогичное распределение можно построить по платформам (iOS/Android), предлагаю сделать это самостоятельно.
Также на этих данных можно построить так называемую RFM-сегментацию пользователей, прочитать можно тут:

https://www.owox.ru/blog/use-cases/rfm-analysis/

https://en.wikipedia.org/wiki/RFM_(market_research)
    
    